<a href="https://colab.research.google.com/github/HabibaAbderrahim/Test_SOTA_2D_pose/blob/main/YOLVSHOURGLASS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/open-mmlab/mmpose/blob/dev-1.x/demo/MMPose_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MMPose Tutorial

Welcome to MMPose colab tutorial! In this tutorial, we will show you how to

- install MMPose 1.x
- perform inference with an MMPose model
- train a new mmpose model with your own datasets

Let's start!

## Install MMPose

We recommend to use a conda environment to install mmpose and its dependencies. And compilers `nvcc` and `gcc` are required.

In [ ]:
# check NVCC version
!nvcc -V

# check GCC version
!gcc --version

# check python in conda environment
!which python

In [ ]:
# install dependencies: (if your colab has CUDA 11.8)
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
# install MMEngine, MMCV and MMDetection using MIM
%pip install -U openmim
!mim install mmengine
!mim install "mmcv>=2.0.0"
!mim install "mmdet>=3.0.0"

In [ ]:
# for better Colab compatibility, install xtcocotools from source
%pip install git+https://github.com/jin-s13/xtcocoapi

In [ ]:
!git clone https://github.com/open-mmlab/mmpose.git
# The master branch is version 1.x
%cd mmpose
%pip install -r requirements.txt
%pip install -v -e .
# "-v" means verbose, or more output
# "-e" means installing a project in editable mode,
# thus any local modifications made to the code will take effect without reinstallation.

In [7]:
# Check Pytorch installation
import torch, torchvision

print('torch version:', torch.__version__, torch.cuda.is_available())
print('torchvision version:', torchvision.__version__)

# Check MMPose installation
import mmpose

print('mmpose version:', mmpose.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version

print('cuda version:', get_compiling_cuda_version())
print('compiler information:', get_compiler_version())

torch version: 2.1.0+cu118 True
torchvision version: 0.16.0+cu118
mmpose version: 1.2.0
cuda version: 11.8
compiler information: GCC 9.3


In [10]:
%cd mmpose


/content/mmpose/mmpose


###TEST HOURGLASS KEYPOINTS VS YOLOX RESULTS

### HOURGLASS

In [11]:
from mmcv.image import imread

from mmpose.apis import inference_topdown, init_model
from mmpose.registry import VISUALIZERS
from mmpose.structures import merge_data_samples

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
#model_cfg = '/content/mmpose/configs/body_2d_keypoint/yoloxpose/coco/yoloxpose_l_8xb32-300e_coco-640.py'

#ckpt = 'https://download.openmmlab.com/mmpose/v1/body_2d_keypoint/yolox_pose/yoloxpose_m_8xb32-300e_coco-640-84e9a538_20230829.pth'
model_cfg ='/content/mmpose/configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_hourglass52_8xb32-210e_coco-256x256.py'
ckpt='https://download.openmmlab.com/mmpose/top_down/hourglass/hourglass52_coco_256x256-4ec713ba_20200709.pth'
device = 'cuda'

# init model
model = init_model(model_cfg, ckpt, device=device)

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/top_down/hourglass/hourglass52_coco_256x256-4ec713ba_20200709.pth


Downloading: "https://download.openmmlab.com/mmpose/top_down/hourglass/hourglass52_coco_256x256-4ec713ba_20200709.pth" to /root/.cache/torch/hub/checkpoints/hourglass52_coco_256x256-4ec713ba_20200709.pth
/content/mmpose/mmpose/datasets/datasets/utils.py:102: UserWarning: The metainfo config file "configs/_base_/datasets/coco.py" does not exist. A matched config file "/content/mmpose/mmpose/.mim/configs/_base_/datasets/coco.py" will be used instead.
  warnings.warn(


In [22]:
img_path = '/content/drive/MyDrive/3D human pose estimation/000000000785.jpg'

In [23]:
# inference on a single image
batch_results = inference_topdown(model, img_path)

In [24]:
pred_instances = batch_results[0].pred_instances

In [25]:
from mmpose.apis import visualize

pred_instances = batch_results[0].pred_instances

keypoints = pred_instances.keypoints
keypoint_scores = pred_instances.keypoint_scores

print(keypoints)

[[[366.875  84.375]
  [373.125  78.125]
  [360.625  78.125]
  [385.625  78.125]
  [354.375  78.125]
  [404.375 109.375]
  [360.625 121.875]
  [435.625 140.625]
  [348.125 159.375]
  [441.875 159.375]
  [304.375 178.125]
  [429.375 203.125]
  [404.375 209.375]
  [429.375 290.625]
  [373.125 271.875]
  [473.125 359.375]
  [404.375 340.625]]]


In [ ]:
# merge results as a single data sample
results = merge_data_samples(batch_results)

# build the visualizer
visualizer = VISUALIZERS.build(model.cfg.visualizer)

# set skeleton, colormap and joint connection rule
visualizer.set_dataset_meta(model.dataset_meta)

img = imread(img_path, channel_order='rgb')

# visualize the results
visualizer.add_datasample(
    'result',
    img,
    data_sample=results,
    show=True)

/usr/local/lib/python3.10/dist-packages/mmengine/utils/manager.py:113: UserWarning: <class 'mmpose.visualization.local_visualizer.PoseLocalVisualizer'> instance named of visualizer has been created, the method `get_instance` should not accept any other arguments
  warnings.warn(


###YOLOX

In [20]:
from mmpose.apis import visualize

pred_instances = batch_results[0].pred_instances

keypoints = pred_instances.keypoints
keypoint_scores = pred_instances.keypoint_scores

print(keypoints)

[[[-8.50177600e+06  2.04660685e+07]
  [-1.28840240e+07  7.36339000e+06]
  [ 7.90586000e+06 -1.76623525e+06]
  ...
  [ 4.38324150e+06 -1.68340635e+07]
  [-6.05544950e+06  1.53340875e+07]
  [-6.28978500e+06 -1.54589895e+07]]

 [[ 4.76717281e+05  1.65536595e+07]
  [-3.62774475e+06  1.01257455e+07]
  [ 1.78393540e+07 -2.26360525e+06]
  ...
  [-1.15513950e+07 -1.53805775e+07]
  [-1.10824300e+06  1.45640545e+07]
  [-9.96435700e+06 -2.39875955e+07]]

 [[-8.50188400e+06  1.22330365e+07]
  [-1.46636720e+07  1.63735865e+07]
  [ 3.33521720e+07 -5.14935850e+06]
  ...
  [-2.03073760e+07 -4.67037750e+06]
  [-5.62667600e+06  2.20977765e+07]
  [-2.69850520e+07 -2.98089495e+07]]

 ...

 [[ 1.31833584e+08 -6.58273260e+08]
  [-3.12805376e+08 -3.01893580e+08]
  [-9.33685440e+08  2.54150468e+08]
  ...
  [ 5.60738048e+08 -3.13409964e+08]
  [ 1.28426752e+09 -1.11230974e+09]
  [-1.02257389e+09  5.86678676e+08]]

 [[ 1.49580224e+08 -6.12519660e+08]
  [-3.58107008e+08 -3.17348396e+08]
  [-8.81701632e+08  2.2400